In [21]:
function _four_points_2d!(points::AbstractVector{<:AbstractVector{N}}) where {N<:Real}
    A, B, C, D = points[1], points[2], points[3], points[4]
    tri_ABC = right_turn(A, B, C)
    tri_ABD = right_turn(A, B, D)
    tri_BCD = right_turn(B, C, D)
    tri_CAD = right_turn(C, A, D)
    key = 0
    if tri_ABC > zero(N)
        key = key + 1000
    end
    if tri_ABD > zero(N)
        key = key + 100
    end
    if tri_BCD > zero(N)
        key = key + 10
    end
    if tri_CAD > zero(N)
        key = key + 1
    end
    
    function collinear_case(A, B, C, D)
        # A, B and C collinear, D is the extra point
        if isapprox(A[1], B[1]) && isapprox(B[1], C[1]) && isapprox(C[1], A[1])
            # points are approximately equal in their first component
            if isapprox(A[2], B[2]) && isapprox(B[2], C[2]) && isapprox(C[2], A[2])
                # the three points are approximately equal
                points[1], points[2] = A, D
                pop!(points)
                pop!(points)
                return _two_points_2d!(points)
            else
                # assign the points with max and min value in their second component to the
                # firsts points and the extra point to the third place, then pop the point that was in the middle
                points[1], points[2], points[3] = points[argmin([A[2], B[2], C[2]])], points[argmax([A[2], B[2], C[2]])], D
                pop!(points)
            end
        else
            # assign the points with max and min value in their first component to the
            # firsts points and the extra point to the third place, then pop the point that was in the middle
            points[1], points[2], points[3] = points[argmin([A[1], B[1], C[1]])], points[argmax([A[1], B[1], C[1]])], D
            pop!(points)
        end
        return _three_points_2d!(points)
    end
    
    if isapproxzero(tri_ABC)
        return collinear_case(A, B, C, D)
    end
    if isapproxzero(tri_ABD)
        return collinear_case(A, B, D, C)
    end
    if isapproxzero(tri_BCD)
        return collinear_case(B, C, D, A)
    end
    if isapproxzero(tri_CAD)
        return collinear_case(C, A, D, B)
    end
    
    # ABC  ABD  BCD  CAD  hull
    # ------------------------
    #  +    +    +    +   ABC
    #  +    +    +    -   ABCD
    #  +    +    -    +   ABDC
    #  +    +    -    -   ABD
    #  +    -    +    +   ADBC
    #  +    -    +    -   BCD
    #  +    -    -    +   CAD
    #  +    -    -    -   [should not happen]
    #  -    +    +    +   [should not happen]
    #  -    +    +    -   ACD
    #  -    +    -    +   DCB
    #  -    +    -    -   DACB
    #  -    -    +    +   ADB
    #  -    -    +    -   ACDB
    #  -    -    -    +   ADCB
    #  -    -    -    -   ACB
    if key == 1111
        points[1], points[2], points[3] = A, B, C # +    +    +    +   ABC
        pop!(points)
    elseif key == 1110
        points[1], points[2], points[3], points[4] = A, B, C, D # +    +    +    -   ABCD
    elseif key == 1101
        points[1], points[2], points[3], points[4] = A, B, D, C #  +    +    -    +   ABDC
    elseif key == 1100
        points[1], points[2], points[3] = A, B, D #  +    +    -    -   ABD
        pop!(points)
    elseif key == 1011
        points[1], points[2], points[3], points[4] = A, D, B, C #  +    -    +    +   ADBC
    elseif key == 1010
        points[1], points[2], points[3] = B, C, D #  +    -    +    -   BCD
        pop!(points)
    elseif key == 1001
        points[1], points[2], points[3] = C, A, D #  +    -    -    +    CAD
        pop!(points)
    elseif key == 0110
        points[1], points[2], points[3] = A, C, D #  -    +    +    -   ACD
        pop!(points)
    elseif key == 0101
        points[1], points[2], points[3] = D, C, B #  -    +    -    +   DCB
        pop!(points)
    elseif key == 0100
        points[1], points[2], points[3], points[4] = D, A, C, B #  -    +    -    -   DACB
    elseif key == 0011
        points[1], points[2], points[3] = A, D, B #  -    -    +    +   ADB
        pop!(points)
    elseif key == 0010
        points[1], points[2], points[3], points[4] = A, C, D, B #  -    -    +    -   ACDB
    elseif key == 0001
        points[1], points[2], points[3], points[4] = A, D, C, B #  -    -    -    +   ADCB
    elseif key == 0000
        points[1], points[2], points[3] = A, C, B #  -    -    -    -   ACB
        pop!(points)
    else
        @assert false "unexpected case in convex_hull"
    end
    return points
end

_four_points_2d! (generic function with 1 method)

In [193]:
using LazySets: sign_cadlag, right_turn
const hull4_table = [
    1,2,3,0,1,2,3,0,1,2,4,3,1,2,3,0,1,2,3,0,1,2,4,0,1,2,3,4,1,2,4,0,1,2,4,0,
    1,2,3,0,1,2,3,0,1,4,3,0,1,2,3,0,0,0,0,0,0,0,0,0,2,3,4,0,0,0,0,0,0,0,0,0,
    1,4,2,3,1,4,3,0,1,4,3,0,2,3,4,0,0,0,0,0,0,0,0,0,2,3,4,0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,2,4,3,0,0,0,0,0,0,0,0,0,1,2,4,0,1,3,4,0,1,2,4,0,1,2,4,0,
    0,0,0,0,0,0,0,0,1,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,4,0,0,0,0,0,0,0,0,0,
    1,4,2,0,1,4,2,0,1,4,3,0,1,4,2,0,0,0,0,0,0,0,0,0,2,3,4,0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,2,4,3,0,0,0,0,0,0,0,0,0,2,4,3,0,1,3,4,0,1,3,4,0,1,3,2,4,
    0,0,0,0,0,0,0,0,2,4,3,0,0,0,0,0,0,0,0,0,1,3,2,0,1,3,4,0,1,3,2,0,1,3,2,0,
    1,4,2,0,1,4,2,0,1,4,3,2,1,4,2,0,1,3,2,0,1,3,2,0,1,3,4,2,1,3,2,0,1,3,2,0
    ]

function convex_hull4(points)
    A, B, C, D = points[1], points[2], points[3], points[4]
    p = copy(points)
    abc = Int(1 - sign_cadlag(right_turn(A, B, C)))
    abd = Int(1 - sign_cadlag(right_turn(A, B, D)))
    cad = Int(1 - sign_cadlag(right_turn(C, A, D)))
    bcd = Int(1 - sign_cadlag(right_turn(B, C, D)))

    t = Int(4 * (bcd + 3*cad + 9*abd + 27*abc))
    (hull4_table[t+1] == 0) ? pop!(points) : (points[1] = p[hull4_table[t+1]])
    (hull4_table[t+2] == 0) ? pop!(points) : (points[2] = p[hull4_table[t+2]])
    (hull4_table[t+3] == 0) ? pop!(points) : (points[3] = p[hull4_table[t+3]])
    (hull4_table[t+4] == 0) ? pop!(points) : (points[4] = p[hull4_table[t+4]])
    return points
end

ErrorException: cannot declare hull4_table constant; it already has a value

In [198]:
N = Float64
A = N[1, 0]
B = N[1, 1]
C = N[-1, 1]
D = N[-1, 0]
points = [A, B, C, D]
@btime convex_hull4(copy($points))

  693.196 ns (2 allocations: 224 bytes)


4-element Array{Array{Float64,1},1}:
 [1.0, 0.0] 
 [1.0, 1.0] 
 [-1.0, 1.0]
 [-1.0, 0.0]

In [189]:
using BenchmarkTools
using LazySets: convex_hull!
N = Float64
A = N[0, 1]
B = N[-1, -1]
C = N[1, -1]
D = N[0, 0]
expr = [A, B, C]
points = [A, C, B, D]
@btime convex_hull4(copy($points))

  597.771 ns (2 allocations: 224 bytes)


3-element Array{Array{Float64,1},1}:
 [0.0, 1.0]  
 [-1.0, -1.0]
 [1.0, -1.0] 

In [199]:
@btime convex_hull!(copy($points))

  485.359 ns (8 allocations: 544 bytes)


4-element Array{Array{Float64,1},1}:
 [-1.0, 0.0]
 [1.0, 0.0] 
 [1.0, 1.0] 
 [-1.0, 1.0]